In [2]:
!pip install --quiet ultralytics

from ultralytics import YOLO
import cv2

model = YOLO('yolov8n.pt')


# Mostrando apenas o rastreamento de objetos q se movem


In [24]:
from ultralytics import YOLO
import cv2
from collections import defaultdict
import numpy as np

cap = cv2.VideoCapture("/home/aprigio/Downloads/Faculdade/Topicos PDI/Projeto TPDI/video1.mp4")

# Usa modelo da Yolo
model = YOLO("yolov8n.pt")

track_history = defaultdict(lambda: [])
seguir = True
deixar_rastro = True

while True:
    success, img = cap.read()

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Pule este frame para melhorar o desempenho

    if success:
        if seguir:
            results = model.track(img, persist=True)
        else:
            results = model(img)

        # Process results list
        for result in results:
            if seguir and deixar_rastro:
                try:
                    person_class_index = 0  # Índice da classe "person" na YOLOv8 é geralmente 0
                    boxes = result.boxes.xywh.cpu()
                    track_ids = result.boxes.id.int().cpu().tolist()
                    class_ids = result.boxes.cls.int().cpu().tolist()

                    # Plot the tracks
                    for box, track_id, class_id in zip(boxes, track_ids, class_ids):
                        if class_id == person_class_index:
                            x, y, w, h = box
                            track = track_history[track_id]
                            track.append((float(x), float(y)))  # x, y center point
                            if len(track) > 30:  # retain 30 tracks for 30 frames
                                track.pop(0)

                            # Draw the tracking lines
                            points = np.array(track).astype(np.int32).reshape((-1, 1, 2))
                            cv2.polylines(img, [points], isClosed=False, color=(230, 0, 0), thickness=5)
                except:
                    pass

        cv2.imshow("Tela", img)

    k = cv2.waitKey(1)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("desligando")



0: 384x640 32 chairs, 4 dining tables, 1 tv, 1 laptop, 62.4ms
Speed: 1.8ms preprocess, 62.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 32 chairs, 4 dining tables, 1 tv, 1 laptop, 64.4ms
Speed: 4.9ms preprocess, 64.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 65.4ms
Speed: 2.0ms preprocess, 65.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 63.0ms
Speed: 5.5ms preprocess, 63.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 61.6ms
Speed: 3.4ms preprocess, 61.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 62.1ms
Speed: 2.5ms preprocess, 62.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 chairs, 61.2ms
Speed: 3.0ms preprocess, 61.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 chairs, 69.6ms
Speed: 3.3ms preproce

Speed: 3.1ms preprocess, 61.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 chairs, 59.7ms
Speed: 2.0ms preprocess, 59.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 chairs, 54.6ms
Speed: 3.6ms preprocess, 54.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 chairs, 62.1ms
Speed: 2.4ms preprocess, 62.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 chairs, 55.1ms
Speed: 3.0ms preprocess, 55.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 chairs, 54.9ms
Speed: 1.9ms preprocess, 54.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 82.6ms
Speed: 4.1ms preprocess, 82.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 63.7ms
Speed: 3.5ms preprocess, 63.7ms inference, 0.8

Speed: 2.8ms preprocess, 62.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 64.1ms
Speed: 2.6ms preprocess, 64.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 58.3ms
Speed: 3.8ms preprocess, 58.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 59.2ms
Speed: 6.5ms preprocess, 59.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 73.1ms
Speed: 13.2ms preprocess, 73.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 60.1ms
Speed: 2.2ms preprocess, 60.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 57.2ms
Speed: 1.8ms preprocess, 57.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 63.2ms
Speed: 3.5ms preprocess, 63.2ms inferenc

# Detecção 


In [22]:
from ultralytics import YOLO
import cv2
from collections import defaultdict
import numpy as np

cap = cv2.VideoCapture("/home/aprigio/Downloads/Faculdade/Topicos PDI/Projeto TPDI/video1.mp4")

# Usa modelo da Yolo
model = YOLO("yolov8n.pt")

track_history = defaultdict(lambda: [])
seguir = True
deixar_rastro = True


while True:
    success, img = cap.read()

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Pule este frame para melhorar o desempenho

    if success:
        if seguir:
            results = model.track(img, persist=True)
        else:
            results = model(img)

        # Process results list
        for result in results:
            img = result.plot()
            if seguir and deixar_rastro:
                try:
                    person_class_index = 0  # Índice da classe "person" na YOLOv8 é geralmente 0
                    boxes = result.boxes.xywh.cpu()
                    track_ids = result.boxes.id.int().cpu().tolist()
                    class_ids = result.boxes.cls.int().cpu().tolist()

                    # Plot the tracks
                    for box, track_id, class_id in zip(boxes, track_ids, class_ids):
                        if class_id == person_class_index:
                            x, y, w, h = box
                            track = track_history[track_id]
                            track.append((float(x), float(y)))  # x, y center point
                            if len(track) > 30:  # retain 30 tracks for 30 frames
                                track.pop(0)

                            # Draw the tracking lines
                            points = np.array(track).astype(np.int32).reshape((-1, 1, 2))
                            cv2.polylines(img, [points], isClosed=False, color=(230, 0, 0), thickness=5)
                except:
                    pass

        cv2.imshow("Tela", img)

    k = cv2.waitKey(1)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("desligando")



0: 384x640 32 chairs, 4 dining tables, 1 tv, 1 laptop, 95.1ms
Speed: 3.0ms preprocess, 95.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 32 chairs, 4 dining tables, 1 tv, 1 laptop, 73.9ms
Speed: 4.7ms preprocess, 73.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 63.6ms
Speed: 2.6ms preprocess, 63.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 62.7ms
Speed: 2.3ms preprocess, 62.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 69.0ms
Speed: 1.9ms preprocess, 69.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 82.8ms
Speed: 17.9ms preprocess, 82.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 chairs, 95.1ms
Speed: 3.0ms preprocess, 95.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 chairs, 73.2ms
Speed: 4.3ms preproc

Speed: 2.0ms preprocess, 62.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 chairs, 66.4ms
Speed: 2.4ms preprocess, 66.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 chairs, 62.1ms
Speed: 1.8ms preprocess, 62.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 chairs, 65.6ms
Speed: 2.2ms preprocess, 65.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 chairs, 68.8ms
Speed: 3.0ms preprocess, 68.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 chairs, 61.0ms
Speed: 2.2ms preprocess, 61.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 58.3ms
Speed: 2.4ms preprocess, 58.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 63.9ms
Speed: 1.8ms preprocess, 63.9ms inference, 0.9

Speed: 7.2ms preprocess, 53.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 62.6ms
Speed: 2.8ms preprocess, 62.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 72.2ms
Speed: 3.6ms preprocess, 72.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 101.1ms
Speed: 6.0ms preprocess, 101.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 95.4ms
Speed: 6.0ms preprocess, 95.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 65.7ms
Speed: 7.1ms preprocess, 65.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 73.6ms
Speed: 2.7ms preprocess, 73.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 chairs, 62.0ms
Speed: 3.3ms preprocess, 62.0ms inferen